In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import glob
import os
import re
import os
from datetime import datetime


In [ ]:
import os, glob, re

carpeta_xml = "/content/sample_data/xml"  # Cambia por tu carpeta
archivos = glob.glob(os.path.join(carpeta_xml, "*.[xX][mM][lL]"))

# Definir namespaces posibles
namespaces = {
    "3.3": {"cfdi": "http://www.sat.gob.mx/cfd/3", "tfd": "http://www.sat.gob.mx/TimbreFiscalDigital"},
    "4.0": {"cfdi": "http://www.sat.gob.mx/cfd/4", "tfd": "http://www.sat.gob.mx/TimbreFiscalDigital"}
}

def extraer_dato(texto, patron):
    if not texto:
        return ""
    match = re.search(patron, texto, re.IGNORECASE)
    return match.group(1).strip() if match else ""

datos = []
no_procesados = []

for archivo in archivos:
    try:
        tree = ET.parse(archivo)
        root = tree.getroot()

        # Detectar versión CFDI
        version = root.attrib.get("Version") or root.attrib.get("version", "")
        ns = namespaces.get(version, namespaces["4.0"])  # default CFDI 4.0

        # Fecha de modificación del archivo
        fecha_mod = datetime.fromtimestamp(os.path.getmtime(archivo)).strftime("%Y-%m-%d %H:%M:%S")

        # ---- Datos CFDI ----
        fecha_completa = root.attrib.get("Fecha", "") or root.attrib.get("fecha", "")
        fecha_simple = fecha_completa.split("T")[0] if "T" in fecha_completa else fecha_completa

        folio = root.attrib.get("Folio", "") or root.attrib.get("folio", "")
        folio_ult5 = folio[-5:] if folio else ""

        subtotal = root.attrib.get("SubTotal", "") or root.attrib.get("subTotal", "")
        total = root.attrib.get("Total", "") or root.attrib.get("total", "")

        emisor = root.find('cfdi:Emisor', ns)
        nombre_emisor = emisor.attrib.get("Nombre") if emisor is not None else emisor.attrib.get("nombre", "") if emisor is not None else ""
        rfc_emisor = emisor.attrib.get("Rfc") if emisor is not None else emisor.attrib.get("rfc", "") if emisor is not None else ""

        timbre = root.find('.//tfd:TimbreFiscalDigital', ns)
        uuid = timbre.attrib.get("UUID") if timbre is not None else timbre.attrib.get("uuid", "") if timbre is not None else ""
        uuid_ult5 = uuid[-5:] if uuid else ""

        # ---- Extraer textos de Comentario y Texto ----
        contrato = ""
        partida = ""
        clabe = ""
        vigencia = ""

        # 1️⃣ Revisar <Comentario>
        for comentario in root.findall(".//Comentario"):
            valor = "".join(comentario.findtext("valor", default=""))
            if not contrato:
                contrato = extraer_dato(valor, r"CONTRATO\s+([A-Z0-9\-\/]+)")
            if not partida:
                partida = extraer_dato(valor, r"PARTIDA\s+PRESUPUESTAL\s+([\d\s+y,]+)")
            if not clabe:
                clabe = extraer_dato(valor, r"CLABE\s+INTERBANCARIA\s+(\d+)")
            if not vigencia:
                vigencia = extraer_dato(valor, r"VIGENCIA\s+DEL\s+CONTRATO\s+(\d{2}/\d{2}/\d{4}\s+AL\s+\d{2}/\d{2}/\d{4})")

        # 2️⃣ Revisar <Texto>
        for texto in root.findall(".//Texto"):
            codigo = texto.findtext("Codigo", "").strip().upper()
            contenido = texto.findtext("Contenido", "")
            if codigo == "CONTRATO" and not contrato:
                contrato = contenido.strip()
            if "PARTIDA PRESUPUESTAL" in codigo or "PRESUPUESTAL" in contenido.upper():
                if not partida:
                    partida = extraer_dato(contenido, r"PARTIDA\s+PRESUPUESTAL\s+([\d\s+y,]+)")
            if "CLABE" in codigo or "CLABE" in contenido.upper():
                if not clabe:
                    clabe = extraer_dato(contenido, r"(\d{18})")
            if "VIGENCIA" in codigo or "VIGENCIA" in contenido.upper():
                if not vigencia:
                    vigencia = contenido.strip()

        # 3️⃣ Buscar en todo el árbol
        for elem in root.iter():
            if elem.text:
                valor = elem.text.strip()

                if not contrato:
                    contrato = extraer_dato(valor, r"CONTRATO\s+([A-Z0-9\-\/]+)")
                if not partida:
                    partida = extraer_dato(valor, r"PARTIDA\s+PRESUPUESTAL\s+([\d\s+y,]+)")
                if not clabe:
                    clabe = extraer_dato(valor, r"CLABE\s+INTERBANCARIA\s+(\d+)")
                if not vigencia:
                    vigencia = extraer_dato(valor, r"VIGENCIA\s+DEL\s+CONTRATO\s+(\d{2}/\d{2}/\d{4}\s+AL\s+\d{2}/\d{2}/\d{4})")

        # 4️⃣ Revisar Addenda (ejemplo AbbVie)
        for comentario in root.findall(".//cfdi:Addenda//Comentario//valor", ns):
          valor = comentario.text or ""

          if not contrato:
            contrato = extraer_dato(valor, r"CONTRATO\s+([A-Z0-9\-\/]+)")
          if not partida:
            partida = extraer_dato(valor, r"PARTIDA\s+PRESUPUESTAL\s+([\d\s+y,]+)")
          if not clabe:
            clabe = extraer_dato(valor, r"CLABE\s+INTERBANCARIA\s+(\d+)")
          if not vigencia:
            vigencia = extraer_dato(valor, r"VIGENCIA\s+DEL\s+CONTRATO\s+(\d{2}/\d{2}/\d{4}\s+AL\s+\d{2}/\d{2}/\d{4})")




        # Guardar datos
        datos.append({
            "Archivo": os.path.basename(archivo),
            "Version": version,
            "Fecha_Modificacion": fecha_mod,
            "Fecha": fecha_simple,
            "Folio": folio,
            "Folio_Ult5": folio_ult5,
            "Nombre_Emisor": nombre_emisor,
            "RFC_Emisor": rfc_emisor,
            "Subtotal": subtotal,
            "Total": total,
            "UUID": uuid,
            "UUID_Ult5": uuid_ult5,
            "Contrato": contrato,
            "Partida_Presupuestal": partida,
            "Clabe_Interbancaria": clabe,
            "Vigencia_Contrato": vigencia
        })

    except Exception as e:
        print(f"❌ Error procesando {archivo}: {e}")
        no_procesados.append(os.path.basename(archivo))

# Guardar en CSV y Excel
df = pd.DataFrame(datos)
df.to_csv("facturas.csv", index=False, encoding="utf-8-sig")
df.to_excel("facturas.xlsx", index=False)

print(f"✅ Procesados {len(datos)} XML. Guardado en facturas.csv y facturas.xlsx.")
if no_procesados:
    print(f"⚠️ No se pudieron procesar {len(no_procesados)} archivos:")
    for f in no_procesados:
        print("   -", f)

for archivo in archivos:
    try:
        os.remove(archivo)
        print(f"🗑️ Eliminado: {os.path.basename(archivo)}")
    except Exception as e:
        print(f"❌ No se pudo eliminar {os.path.basename(archivo)}: {e}")



✅ Procesados 23 XML. Guardado en facturas.csv y facturas.xlsx.
🗑️ Eliminado: ACC0001122E7FFA 129833.xml
🗑️ Eliminado: 0090949479 pascual.xml
🗑️ Eliminado: ACC0001122E7FFA 129741.xml
🗑️ Eliminado: ACC0001122E7FFA 129740.xml
🗑️ Eliminado: 12128248.XML.XML
🗑️ Eliminado: ACC0001122E7FFA 129745.xml
🗑️ Eliminado: ACC0001122E7FFA 129681.xml
🗑️ Eliminado: 0090946314 PASCUAL.xml
🗑️ Eliminado: ACC0001122E7FFA 129737.xml
🗑️ Eliminado: ACC0001122E7FFA 129743.xml
🗑️ Eliminado: 0090945191 PASCUAL.xml
🗑️ Eliminado: ACC0001122E7FFA 129683.xml
🗑️ Eliminado: 0928114455 AMGEN.XML
🗑️ Eliminado: ACC0001122E7FFA 129748.xml
🗑️ Eliminado: 0928114456 AMGEN.XML
🗑️ Eliminado: ACC0001122E7FFA 129746.xml
🗑️ Eliminado: ACC0001122E7FFA 129742.xml
🗑️ Eliminado: 12128249.XML.XML
🗑️ Eliminado: B9346944345 PFIZER.xml
🗑️ Eliminado: ACC0001122E7FFA 129749.xml
🗑️ Eliminado: 0063306475.xml
🗑️ Eliminado: 0090948303 PASCUAL.xml
🗑️ Eliminado: 0063306476.xml
